# Real Estate Pricing Tier Classification by Thai, Laxmi, and Daniel

### Data Cleaning & Preparation - Thai

In [19]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [23]:
# Load the dataset
real_estate = pd.read_csv('realtor-data.zip.csv')

# Print the first few rows of the original dataset
print("Original dataset:")
print(real_estate.head())

# Print all columns in the original dataset
print("\nAll columns in the original dataset:")
print(real_estate.columns)

# Display the data types of each column
print("\nData types of each column:")
print(real_estate.dtypes)

# Check the dimensions of the dataset
print("\nNumber of rows and columns in the dataset:", real_estate.shape)

# Check for missing values
print("\nMissing values in the dataset:")
print(real_estate.isnull().sum())

# Display all unique values in the 'state' column
print("\nAll states:")
print(real_estate['state'].unique())

# Define the list of states to delete
states_to_delete = ['New Brunswick', 'Puerto Rico', 'Virgin Islands']

# Filter the DataFrame to keep only rows where the 'state' column does not belong to the states to delete
real_estate = real_estate[~real_estate['state'].isin(states_to_delete)]

# Remove rows with missing values
real_estate.dropna(inplace=True)

# Remove irrelevant columns
real_estate.drop(['prev_sold_date'], axis=1, inplace=True)

# Initialize LabelEncoder for 'status' column
encoder = LabelEncoder()
real_estate['status'] = encoder.fit_transform(real_estate['status'])

# Define a dictionary to map old column names to new column names for renaming
column_mapping = {'acre_lot': 'lot_size', 'zip_code': 'zipcode', 'price': 'sales_price'}

# Rename specified columns using the defined mapping
real_estate.rename(columns=column_mapping, inplace=True)

# Display USA states only in the 'state' column
print("\nUSA states only:")
print(real_estate['state'].unique())

# Print cleaned dataset
print("\nCleaned dataset:")
print(real_estate.head())

# Print the last few rows of the cleaned dataset
print("\nLast few rows of the cleaned dataset:")
print(real_estate.tail())

# Print the number of rows in the cleaned dataset
print("\nNumber of rows in the cleaned dataset:", real_estate.shape)

# Print cleaned dataset with a note summarizing the changes made
print("\nCleaned dataset with the following modifications:\n"
      "1. Irrelevant columns 'prev_sold_date' removed.\n"
      "2. Rows with missing values removed.\n"
      "3. Categorical column 'status' converted to numerical.\n"
      "4. Columns renamed using defined mapping.\n"
      "5. Rows corresponding to states not in the US filtered out.")

# Save the cleaned dataset
real_estate.to_csv('cleaned_realtor_data.csv', index=False)

Original dataset:
     status  bed  bath  acre_lot        city        state  zip_code  \
0  for_sale  3.0   2.0      0.12    Adjuntas  Puerto Rico     601.0   
1  for_sale  4.0   2.0      0.08    Adjuntas  Puerto Rico     601.0   
2  for_sale  2.0   1.0      0.15  Juana Diaz  Puerto Rico     795.0   
3  for_sale  4.0   2.0      0.10       Ponce  Puerto Rico     731.0   
4  for_sale  6.0   2.0      0.05    Mayaguez  Puerto Rico     680.0   

   house_size prev_sold_date     price  
0       920.0            NaN  105000.0  
1      1527.0            NaN   80000.0  
2       748.0            NaN   67000.0  
3      1800.0            NaN  145000.0  
4         NaN            NaN   65000.0  

All columns in the original dataset:
Index(['status', 'bed', 'bath', 'acre_lot', 'city', 'state', 'zip_code',
       'house_size', 'prev_sold_date', 'price'],
      dtype='object')

Data types of each column:
status             object
bed               float64
bath              float64
acre_lot          flo

### Exploratory Data Analysis - Laxmi